In [ ]:
%pylab inline

In [1]:
import dataiku
import dataikuapi
client = dataiku.api_client()

In [9]:
client.get_general_settings()

In [ ]:
import os
project_key = os.getenv("DKU_CURRENT_PROJECT_KEY")

In [2]:
project_key = "STAKEHOLDER_OWNERSHIP"

In [10]:
dataset_name = "stakeholders_tbc_prepared"
editlog_ds_name = dataset_name + "_editlog"
original_ds = dataiku.Dataset(dataset_name, project_key)
editlog_ds = dataiku.Dataset(editlog_ds_name, project_key)

In [11]:
original_df = original_ds.get_dataframe()

PermissionError: [Errno 1] Operation not permitted

In [5]:
editlog_ds

Column,Type
date,string
user,string
key,string
column_name,string
value,string


In [6]:
editlog_df = editlog_ds.get_dataframe()

PermissionError: [Errno 1] Operation not permitted

## Create new datasets

In [ ]:
project = client.get_default_project()

In [ ]:
changes_ds_creator = dataikuapi.dss.dataset.DSSManagedDatasetCreationHelper(project, changes_ds_name)
changes_ds_creator.with_store_into(connection="filesystem_managed")
changes_ds_creator.create()

In [ ]:
changes_ds = dataiku.Dataset(changes_ds_name)

In [ ]:
changes_ds.write_schema_from_dataframe(df=original_df)

In [ ]:
editable_ds_creator = dataikuapi.dss.dataset.DSSManagedDatasetCreationHelper(project, editable_ds_name)
editable_ds_creator.with_store_into(connection=connection_name)
editable_ds_creator.create()

In [ ]:
editable_ds = dataiku.Dataset(editable_ds_name)

In [ ]:
editable_ds.write_with_schema(original_df)

## Get table name

In [ ]:
editable_tablename = editable_ds.get_config()["params"]["table"].replace("${projectKey}", project_key).replace("${NODE}", dataiku.get_custom_variables()["NODE"])

## SQL queries

### Update row

In [ ]:
idx = 0
col_id = "sepal_length"
val = 0

In [ ]:
INPUT_KEY = "index"
query = """UPDATE \"{0}\" SET {1}={2}
            WHERE {3}={4}
            RETURNING {1};
            COMMIT;
            """.format(editable_tablename, col_id, val, INPUT_KEY, idx)
print(query)

In [ ]:
from dataiku.core.sql import SQLExecutor2
connection_name = original_ds.get_config()['params']['connection']
executor = SQLExecutor2(connection=connection_name)

In [ ]:
result_df = executor.query_to_df(query)
result_df

### Retrieve lookup columns

In [ ]:
ext_lookup_columns = ["sepal_length", "sepal_width"]
ext_tablename = editable_tablename
ext_key = "index"
ext_idx = "0"

In [ ]:
select_query = "SELECT " \
    + ", ".join(ext_lookup_columns) \
    + " FROM \"" + ext_tablename \
    + "\" WHERE " + ext_key + "=" + ext_idx
select_df = executor.query_to_df(select_query)

In [ ]:
val = select_df.iloc[0][ext_lookup_columns[0]]
print(val)

### Update lookup columns

In [ ]:
update_query = "UPDATE \"" + editable_tablename \
    + "\" SET " + ", ".join([col + "=" + str(select_df.iloc[0][col]) for col in ext_lookup_columns]) \
    + " WHERE " + ext_key + "=" + ext_idx \
    + "; COMMIT;"
print(update_query)

In [ ]:
executor.query_to_df(update_query)

## Create a recipe

In [ ]:
recipe_creator = dataikuapi.dss.recipe.DSSRecipeCreator("CustomCode_replay-edits", "compute_iris_editable", project)
recipe = recipe_creator.create()

In [ ]:
settings = recipe.get_settings()
settings.add_input("input", "iris")
settings.add_input("changes", "iris_changes")
settings.add_output("editable", "iris_editable")
settings.raw_params["customConfig"] = {"key": "index"}
settings.save()

Alternative:

```python
recipe_creator.with_input("iris", role="input")
recipe_creator.with_input("iris_changes", role="changes")
recipe_creator.with_output("iris_editable", role="editable")
```

In [ ]:
recipe = project.get_recipe("compute_iris_editable")

In [ ]:
settings = recipe.get_settings()